In [1]:
from funcs_ssp_study import *

import os
import xarray as xr
import pandas as pd
import geopandas as gpd
import re
import json
from tqdm import tqdm
import cdsapi
import zipfile
import shutil
import requests

# Code

In [2]:
data_directory = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline"
data_subdirectory = os.path.join(data_directory, "raw")
output_subdirectory = os.path.join(data_directory, "processed_monthly")

country_shapes_filepath = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
country_codes_filepath = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP/OECD_country_codes.csv"
alignments_filepath = "/Users/michellewan/PycharmProjects/TraCE/alignments.json"

socioeconomic_directory = "/Users/michellewan/Downloads/OECD_population_GDP"
socioeconomic_ssp_filepath = os.path.join(socioeconomic_directory, "SspDb_country_data_2013-06-12.csv")
population_historical_filepath = os.path.join(socioeconomic_directory, "HISTPOP_22082023151824127.csv")
gdp_historical_filepath = os.path.join(socioeconomic_directory, "DP_LIVE_22082023165117703.csv")

In [3]:
climate_variables = [
    "temperature", 
    "pressure",
    "wind_speed",
    "precipitation"
]

socioeconomic_variables = [
    "GDP|PPP", 
    "Population"
]

atmospheric_variables = [
    "ch4"
]

In [4]:
SSPs = [str(ssp) for ssp in range(1, 5+1)]

In [5]:
frequency = "M"

start_year = 2015
end_year = 2023

## DOWNLOAD

In [ ]:
# Downloading climate data from Copernicus CDS requires a registered account (free)
download_climate_variables_data(data_subdirectory, climate_variables, SSPs, start_year, end_year)

In [ ]:
# Atmospheric SSP data is freely available via ESGF
download_atmospheric_variables_data(data_subdirectory, atmospheric_variables, SSPs)

## PROCESS

In [ ]:
process_netcdf_data(data_subdirectory, output_subdirectory, 
                    country_shapes_filepath, country_codes_filepath, alignments_filepath, 
                    frequency)

Processing files...
ssp1_ch4.nc


Aggregating:   3%|██████▎                                                                                                                                                                               | 9/258 [01:13<33:37,  8.10s/it]

In [ ]:
# Country-level GDP and population SSP data is downloaded from IIASA via a registered account (free)

# Process the socioeconomic datasets
extract_socioeconomic_ssp_data(socioeconomic_ssp_filepath, output_subdirectory, socioeconomic_variables, SSPs, start_year, end_year, frequency)
process_population_historical_data(population_historical_filepath, output_subdirectory, frequency)
process_GDP_historical_data(gdp_historical_filepath, output_subdirectory, frequency)

# Scratch

In [61]:
ds = xr.open_dataset("/Users/michellewan/PycharmProjects/TraCE/data_pipeline/raw/ssp3_wind_speed.nc")

In [63]:
ds = ds.drop("height")

In [64]:
ds

<xarray.Dataset>
Dimensions:      (lat: 143, lon: 144, time: 108, axis_nbounds: 2)
Coordinates:
  * lat          (lat) float32 -90.0 -88.73 -87.46 -86.2 ... 87.46 88.73 90.0
  * lon          (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time         (time) datetime64[ns] 2015-01-16T12:00:00 ... 2023-12-16T12:...
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (time, axis_nbounds) datetime64[ns] ...
    sfcWind      (time, lat, lon) float32 ...
Attributes: (12/51)
    name:                   /ccc/work/cont003/gencmip6/p86caub/IGCM_OUT/IPSLC...
    Conventions:            CF-1.7 CMIP-6.2
    creation_date:          2018-12-18T20:37:47Z
    tracking_id:            hdl:21.14100/3dd4fb30-226c-462f-ae35-92620b659473
    description:            Future scenario with high radiative forcing by th...
    title:                  IPSL-CM6A-LR model output prepared for CMIP6 / Sc...
    ...                     ...
    variant_info:           Each member starts from the corresponding member ...
    variant_label:          r1i1p1f1
    EXPID:                  ssp370
    CMIP6_CV_version:       cv=6.2.3.5-2-g63b123e
    dr2xml_md5sum:          c2dce418e78ca835be1e2ff817c2c403
    model_version:          6.1.8

In [69]:
set(ds["sfcWind"].dims) == set(ds.coords)

True

In [70]:
[variable_name for variable_name in ds.data_vars if set(ds[variable_name].dims) == set(ds.coords)]

['sfcWind']

In [41]:
ds.sel(height=ds['height'].values.min())

KeyError: "no index found for coordinate 'height'"

In [ ]:
country_shapes = gpd.read_file(country_shapes_filepath)

In [ ]:
ds["ch4"]

In [ ]:
agg_ds = aggregate_country_data(ds, country_shapes)

In [ ]:
df = convert_nc_to_csv(agg_ds, "ch4")

In [ ]:
# Extract the necessary data variables
time_steps = agg_ds.time.values
countries = agg_ds.country.values
variable = agg_ds.values

# Reshape the data into a 2D array
# Rows: time steps, Columns: countries
variable_reshaped = variable.transpose(1, 0)

# Create a pandas DataFrame
df = pd.DataFrame(data=variable_reshaped, index=time_steps, columns=countries)

In [ ]:
df.index

In [ ]:
df.index = df.index.astype(str)

In [ ]:
df.index

In [ ]:
import cftime

# Convert the cftime.DatetimeNoLeap objects to standard datetime objects
datetime_index = [cftime.datetime(date.year, date.month, date.day) for date in df.index]


In [ ]:
df.index = pd.to_datetime(df.index.astype(str))
df.resample("Q").mean()

In [7]:
df.index

NameError: name 'df' is not defined